In [ ]:
import pathlib
import tempfile
from tkinter import filedialog
import zipfile
import logging

import pandas as pd
from sqlalchemy import create_engine

In [ ]:
import sys
sys.path.append(r'D:\Keralty scripts\automatizaciones_python\actualizacion_parametricas_upc\database\operaciones_bdoracle.py')
from operaciones_bdoracle import actualizar_datos_oracle, conectar_base_oracle, creacion_tabla_actualizada, obtener_datos_oracle

In [ ]:
# Constantes
NOMBRE_TABLA_PRESTADORES = 'tbl_ope_universo_prestadores'
COLUMNAS_PRESTADORES = [
    'DESCRIPCION PLAN',
    'FORMA CONTRATACION',
    'NUM ID',
    'TIPO ID',
    'TIPO PERSONA',
    'CODIGO SUCURSAL',
    'NOMBRE SUCURSAL',
    'CIUDAD',
    'DESCRIPCION CIUDAD',
    'DEPARTAMENTO',
    'DESCRIPCION ESPECIALIDAD',
    'ESTADO',
    'TIPO CONVENIO',
    'COD HABILITACION SUCURSAL',
    'HABILITACIÓN SEDE SUCURSAL',
    'FECHA INICIO CONVENIO',
    'FECHA FIN CONVENIO',
    'REGIONAL']

In [ ]:
periodo = input('Ingrese el periodo de actualización (YYYYMM): ')
periodo

In [ ]:
ruta_prestadores = filedialog.askopenfilename(title='Seleccione el archivo de prestadores')
ruta_prestadores

In [ ]:
carpeta_regionales = pathlib.Path(
    r'G:\.shortcut-targets-by-id\1wT-pRaNOECz6KC5hndveeLOIGY381o4T\Alteryx\Proyectos\154._Tableros_RIPS\03.Salidas\_Tb_Regiones_.csv')

In [ ]:
with zipfile.ZipFile(ruta_prestadores, 'r') as zip_file:
     with tempfile.TemporaryDirectory() as temp_dir:
        zip_file.extractall(temp_dir)
        archivos = pathlib.Path(temp_dir).rglob('*.xlsx')  # Esto es un generador
        try:
            # Obtenemos el primer archivo encontrado
            archivo = next(archivos, None)
            if archivo is None:        logging.error(
                        f'No se encontró ningún archivo .xlsx en el ZIP')
            else:
                    logging.info(f'Leyendo archivo {archivo}')
                    df_prestadores = pd.read_excel(archivo, sheet_name='E.P.S Sanitas',
                                                   skiprows=2, dtype='str', usecols=COLUMNAS_PRESTADORES)
        except Exception as e:
                logging.error(f'Error al leer el archivo de prestadores: {e}')

In [ ]:
df_regionales = pd.read_csv(carpeta_regionales, sep='|', dtype='str')

# Preparacion para cruce de regionales
df_regionales['regional_adaptada'] = df_regionales['Regional'].str.replace(
        ' ', '_', 1)
df_regionales['nombre_region'] = df_regionales['regional_adaptada'].str.split(
        pat=' ', n=1).str[1]

# Mantener unicos por nombre de regional y regional adaptada
df_regionales.drop_duplicates(
        subset=['nombre_region', 'regional_adaptada'], inplace=True)

In [ ]:
# Cruce de regionales con prestadores
df_prestadores = pd.merge(df_prestadores, df_regionales[[
                              'regional_adaptada', 'nombre_region']], left_on='REGIONAL', right_on='nombre_region', how='inner')
df_prestadores.drop(columns=['nombre_region', 'REGIONAL'], inplace=True)
df_prestadores.rename(
        columns={'regional_adaptada': 'REGIONAL'}, inplace=True)

In [ ]:
# Procesamiento prestadores
df_prestadores['CODIGO SUCURSAL23'] = df_prestadores['CODIGO SUCURSAL']
df_prestadores['AVICENA'] = ''
df_prestadores['NIT2'] = df_prestadores['TIPO ID'].str.slice(
        0, 1) + df_prestadores['NUM ID']
df_prestadores['MUNICIPIO AUTORIZADO'] = ''
df_prestadores['COD_HABILITACION'] = df_prestadores['COD HABILITACION SUCURSAL'] + \
df_prestadores['HABILITACIÓN SEDE SUCURSAL']


In [ ]:
# Eliminar columnas no necesarias
df_prestadores.drop(columns=['HABILITACIÓN SEDE SUCURSAL',
                        'COD HABILITACION SUCURSAL', 'TIPO ID'], inplace=True)


In [ ]:
# Define el nombre de las columnas
column_renames = {
        "FORMA CONTRATACION": "RELACION EPS",
        "NUM ID": "NIT",
        "TIPO PERSONA": "TIPO_PERSONA",
        "CODIGO SUCURSAL": "CODIGO SUCURSAL22",
        "NOMBRE SUCURSAL": "PRESTADOR",
        "CIUDAD": "COD_CIUDAD",
        "DESCRIPCION CIUDAD": "CIUDAD",
        "DESCRIPCION ESPECIALIDAD": "GRUPO_SERVICIO",
        "ESTADO": "Estado Actual",
        "FECHA INICIO CONVENIO": "INICIO_CONTRATO",
        "FECHA FIN CONVENIO": "FIN_VIGENCIA",
        "COD_HABILITACION": "CODIGO DE HABILITACION2",
}

In [ ]:
df_prestadores.columns

In [ ]:
engine = conectar_base_oracle()

In [ ]:
print(f'Columnas prestadores: {df_prestadores.columns}')
    
df_prestadores.rename(columns=column_renames, inplace=True)

print(f'Columnas prestadores renombradas: {df_prestadores.columns}')

# creacion_tabla_actualizada(
#         engine, df_prestadores, NOMBRE_TABLA_PRESTADORES, periodo)

In [ ]:
    
# obtener universo prestadores
df_universo_prestadores = obtener_datos_oracle(engine, NOMBRE_TABLA_PRESTADORES)

df_universo_prestadores

In [ ]:
df_prestadores[df_prestadores['NIT'] == '15242933']

In [ ]:
df_universo_prestadores.loc[(~df_universo_prestadores['nit'].isin(df_prestadores['NIT'])) & (df_universo_prestadores['Estado Actual'] != 'FINALIZADO')].shape

In [ ]:
df_universo_prestadores.columns

In [ ]:
# renombrar columnas a lower case cuando la columna no tiene espacios
for col in df_prestadores.columns:
    if ' ' not in col:
        df_prestadores.rename(columns={col: col.lower()}, inplace=True)
df_prestadores.columns

In [ ]:
df_universo_prestadores

In [ ]:
# poner marca df_universo_prestadores['Estado Actual'] = 'FINALIZADO' si no se encuentra en df_prestadores
df_universo_prestadores.loc[~df_universo_prestadores['nit'].isin(df_prestadores['nit']), 'Estado Actual'] = 'FINALIZADO'

In [ ]:
df_universo_prestadores['Estado Actual'].value_counts()

In [ ]:
df_universo_prestadores_finalizados = df_universo_prestadores.loc[df_universo_prestadores['Estado Actual'] == 'FINALIZADO']
df_universo_prestadores_finalizados.shape

In [ ]:
df_universo_prestadores_completos = pd.concat([df_prestadores, df_universo_prestadores_finalizados], ignore_index=True)
df_universo_prestadores_completos.head()

In [ ]:
df_universo_prestadores_completos.shape

In [ ]:
df_universo_prestadores_completos[df_universo_prestadores_completos['CODIGO DE HABILITACION2'] == '730010242403']

In [ ]:
df_universo_prestadores_completos.drop_duplicates(inplace=True)

In [ ]:
df_universo_prestadores_completos['tipo_persona'] = df_universo_prestadores_completos['tipo_persona'].str.upper()
df_universo_prestadores_completos['tipo_persona'].value_counts()

In [ ]:
df_universo_prestadores_completos[df_universo_prestadores_completos.duplicated()]

In [ ]:
actualizar_datos_oracle(engine, df_universo_prestadores_completos, NOMBRE_TABLA_PRESTADORES)

In [ ]:
NOMBRE_TABLA_PRESTADORES